<a href="https://colab.research.google.com/github/marcgym2/MachineLearning/blob/main/Tarea8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tarea 8 (10 puntos)
- Revisa en la literatura las métricas de desempeño que se utilizan en tu problema.
- Elige al menos una métrica para aplicar a tus modelos.
- Reporta estos hallazgos en el marco teórico de tu artículo.

In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers

In [1]:
# Cargar los datos
from google.colab import drive
drive.mount("/content/drive")
df_measures = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/measures_v2.csv")

Mounted at /content/drive


In [43]:
# Definir variables predictoras y variable a predecir
x_vars = ["u_q", "coolant", "ambient", "u_d", "motor_speed", "i_q", "i_d"]
y_vars = ["pm", "torque", "stator_yoke", "stator_tooth", "stator_winding"]

In [44]:
# Dividir los datos en conjunto de entrenamiento y prueba
train_size = int(len(df_measures) * 0.7)
train_set = df_measures.iloc[:train_size]
test_set = df_measures.iloc[train_size:]

In [45]:
# Escalar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
train_set[x_vars + y_vars] = scaler.fit_transform(train_set[x_vars + y_vars])
test_set[x_vars + y_vars] = scaler.transform(test_set[x_vars + y_vars])

/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [46]:
# Definir el modelo de red neuronal
model = Sequential()

# Añadir las capas ocultas
model.add(Dense(128, input_dim=len(x_vars), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

In [47]:
# Añadir la capa de salida
model.add(Dense(len(y_vars), activation='linear', 
                kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)))

In [48]:
# Definir el optimizador y la función de pérdida
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse', optimizer=opt)

In [49]:
# Entrenar el modelo
history = model.fit(train_set[x_vars], train_set[y_vars],
                    validation_data=(test_set[x_vars], test_set[y_vars]),
                    epochs=50, batch_size=32)

Epoch 1/50
29112/29112 [==============================] - 101s 3ms/step - loss: 0.0102 - val_loss: 0.0102
Epoch 2/50
29112/29112 [==============================] - 92s 3ms/step - loss: 0.0084 - val_loss: 0.0110
Epoch 3/50
29112/29112 [==============================] - 91s 3ms/step - loss: 0.0080 - val_loss: 0.0101
Epoch 4/50
29112/29112 [==============================] - 92s 3ms/step - loss: 0.0078 - val_loss: 0.0098
Epoch 5/50
29112/29112 [==============================] - 96s 3ms/step - loss: 0.0077 - val_loss: 0.0105
Epoch 6/50
29112/29112 [==============================] - 90s 3ms/step - loss: 0.0076 - val_loss: 0.0103
Epoch 7/50
29112/29112 [==============================] - 90s 3ms/step - loss: 0.0075 - val_loss: 0.0101
Epoch 8/50
29112/29112 [==============================] - 92s 3ms/step - loss: 0.0075 - val_loss: 0.0101
Epoch 9/50
29112/29112 [==============================] - 93s 3ms/step - loss: 0.0074 - val_loss: 0.0102
Epoch 10/50
29112/29112 [=============================

In [52]:
# Obtener las predicciones en el conjunto de prueba
predictions = model.predict(test_set[x_vars])

12477/12477 [==============================] - 33s 3ms/step


In [53]:
# Calcular las métricas de evaluación en el conjunto de prueba
mse = np.mean(np.square(predictions - test_set[y_vars]))
rmse = np.sqrt(mse)
mae = np.mean(np.abs(predictions - test_set[y_vars]))
r2 = 1 - (np.sum(np.square(predictions - test_set[y_vars])) / 
          np.sum(np.square(test_set[y_vars] - np.mean(test_set[y_vars], axis=0))))

print("RMSE en el conjunto de prueba:", rmse)
print("R^2 en el conjunto de prueba:", r2)
print("MAE en el conjunto de prueba:", mae)
print("MSE en el conjunto de prueba:", mse)

RMSE en el conjunto de prueba: pm                0.108869
torque            0.036204
stator_yoke       0.094262
stator_tooth      0.115637
stator_winding    0.117066
dtype: float64
R^2 en el conjunto de prueba: pm                0.480450
torque            0.943500
stator_yoke       0.836566
stator_tooth      0.723499
stator_winding    0.650030
dtype: float64
MAE en el conjunto de prueba: pm                0.086587
torque            0.029574
stator_yoke       0.073415
stator_tooth      0.091478
stator_winding    0.093699
dtype: float64
MSE en el conjunto de prueba: pm                0.011853
torque            0.001311
stator_yoke       0.008885
stator_tooth      0.013372
stator_winding    0.013704
dtype: float64
